In [1]:
# Import standard libraries
import os
import sys
import importlib
import datetime as dt
import time
from pathlib import Path
from contextlib import redirect_stdout

# Import data manipulation libraries
import numpy as np
import pandas as pd

# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Import custom modules - NEED WINDOWS OS
from CoEQWAL.imports import AuxFunctions as af, cs3, csPlots, cs_util as util, dss3_functions_reference as dss

### Import Water Year Type Table

In [4]:
df_wy = pd.read_csv("../data/wytypestable.csv")
df_wy

,WATERYEAR,SACindex_0,SJRindex_0,SHASTAindex_0,AmerD893_0,FEATHERindex_0,Trinityindex_0,Amer403030_0,SACindex_1,SJRindex_1,...,Trinityindex_9,Amer403030_9,SACindex_10,SJRindex_10,SHASTAindex_10,AmerD893_10,FEATHERindex_10,Trinityindex_10,Amer403030_10,DriestYrs
0,1920,2,2,1,1,0,3,2,2,2,...,3,2,2,2,1,1,0,3,2,0
1,1921,2,2,1,1,0,3,2,2,2,...,3,2,2,2,1,1,0,3,2,0
2,1922,2,1,1,1,0,4,2,2,2,...,3,2,1,1,1,1,0,4,2,0
3,1923,3,2,3,1,0,4,3,4,3,...,4,3,3,2,3,1,0,4,3,0
4,1924,5,5,4,2,1,5,6,5,5,...,5,6,5,5,4,2,1,5,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1999,1,2,1,1,0,2,0,1,3,...,1,0,1,2,1,1,0,2,0,0
80,2000,2,2,1,1,0,2,0,2,3,...,1,0,2,3,1,1,0,1,0,0
81,2001,4,4,1,2,0,4,0,5,5,...,3,0,4,5,2,2,0,4,0,0
82,2003,2,3,1,1,0,2,0,3,5,...,1,0,3,5,1,1,0,1,0,0


In [5]:
baseline_cols = [col for col in df_wy.columns if col.endswith('_0')]
baseline_cols.append("WATERYEAR")
df_wy_filter = df_wy[baseline_cols]
df_wy_filter.head(5)

,SACindex_0,SJRindex_0,SHASTAindex_0,AmerD893_0,FEATHERindex_0,Trinityindex_0,Amer403030_0,WATERYEAR
0,2,2,1,1,0,3,2,1920
1,2,2,1,1,0,3,2,1921
2,2,1,1,1,0,4,2,1922
3,3,2,3,1,0,4,3,1923
4,5,5,4,2,1,5,6,1924


In [24]:
df = pd.read_csv("../data/EDA_data_04_09_24.csv", header=[0, 1, 2, 3, 4, 5, 6], index_col=0, parse_dates=True)
df = df.loc[:, ~df.columns.get_level_values(6).str.contains('CFS.1')]
var_df = pd.read_csv("EDA_vars_04_09_24.csv")
dss_names = pd.read_csv("dss_names.csv")["0"].tolist()
df.head(5)
var_list = var_df["DSS Part B"].tolist()

In [25]:
var_df

,Unnamed: 0,DSS Part B,DSS Part C
0,0,S_SHSTA,STORAGE
1,1,S_OROVL,STORAGE
2,2,S_TRNTY,STORAGE
3,3,S_MELON,STORAGE
4,4,S_MLRTN,STORAGE
5,5,S_SHSTALEVEL1DV,STORAGE-LEVEL
6,6,S_SHSTALEVEL5DV,STORAGE-LEVEL
7,7,S_OROVLLEVEL1DV,STORAGE-LEVEL
8,8,S_TRNTYLEVEL1DV,STORAGE-LEVEL
9,9,S_MELONLEVEL1DV,STORAGE-LEVEL


## Subset by variable across studies

In [26]:
def create_subset(df, varname):
    """ 
    Filters df to return columns that contain the string varname
    :param df: Dataframe to filter
    :param varname: variable of interest, e.g. S_SHSTA
    """
    filtered_columns = df.columns.get_level_values(1).str.contains(varname)
    return df.loc[:, filtered_columns]

In [27]:
def create_subset_list(df, var_names):
    """ 
    Filters df to return columns that contain any of the strings in var_names.
    :param df: Dataframe to filter.
    :param var_names: List of variables of interest, e.g. ['S_SHSTA', 'S_OROVL'].
    """
    filtered_columns = df.columns.get_level_values(1).str.contains('|'.join(var_names))
    return df.loc[:, filtered_columns]

In [28]:
def compute_annual_means(df, variable_list, study_lst, months=None):
    subset_df = create_subset_list(df, variable_list).iloc[:, study_lst]
    if months is not None:
        subset_df = subset_df[subset_df.index.month.isin(months)]
    sum_df = subset_df.sum(axis=1)
    
    annual_mean = sum_df.resample('Y').mean() / len(study_lst)

    return annual_mean.to_frame(name='Annual Mean')

In [29]:
def compute_mean(df, variable_list, study_lst, months):
    df = compute_annual_means(df, variable_list, study_lst, months)
    return sum(df["Annual Mean"]) / df.shape[0]

In [34]:
fPath = "../visualizations"

In [35]:
SECS_IN_MONTH = 2628000

def convert_cfs_to_taf(df):
    
#    columns_to_convert = [col for col in df.columns if ('DEL' in col[1] or 'NDO' in col[1]) and 'CFS' in col[6]]
    columns_to_convert = [col for col in df.columns if 'DEL' in col[1] and 'CFS' in col[6]]
    for column in columns_to_convert:
        df[column] = (df[column] * SECS_IN_MONTH) / 1000

    new_columns = []
    for col in df.columns:
        if col in columns_to_convert:
            new_col = col[:6] + ('TAF',) + col[7:]  
            new_columns.append(new_col)
        else:
            new_columns.append(col)

    df.columns = pd.MultiIndex.from_tuples(new_columns) 

    return df

In [37]:
df = convert_cfs_to_taf(df)
df

CALLITE                                           \
           C_SJRVER_expl0000 C_WILKNS_expl0000 DEL_CVP_PAG_N_expl0000   
                FLOW-CHANNEL      FLOW-CHANNEL           DELIVERY-CVP   
                        1MON              1MON                   1MON   
                    2020D09E          2020D09E               2020D09E   
                    PER-AVER          PER-AVER               PER-AVER   
                         CFS               CFS                    TAF   
1921-10-31         2505.2790         5427.5645           2.092484e+05   
1921-11-30         1977.8469         4904.4517           0.000000e+00   
1921-12-31         2740.8728         9056.1770           0.000000e+00   
1922-01-31         2577.6514         6841.2305           0.000000e+00   
1922-02-28         5592.3980        16573.0140           0.000000e+00   
..                       ...               ...                    ...   
2003-05-31         3318.8572        18618.2580           1.874032e+06   
2003-06-30         2222.6667         4923.5800           3.628755e+06   
2003-07-31         1147.9255         9365.8700           4.052868e+06   
2003-08-31         1359.1146         5336.4430           2.851372e+06   
2003-09-30         1674.8894         9031.4380           1.627183e+06   

                                                          \
           DEL_CVP_PAG_S_expl0000 DEL_CVP_PRF_S_expl0000   
                     DELIVERY-CVP           DELIVERY-CVP   
                             1MON                   1MON   
                         2020D09E               2020D09E   
                         PER-AVER               PER-AVER   
                              TAF                    TAF   
1921-10-31           2.423343e+06           2.813381e+06   
1921-11-30           1.848376e+06           1.812635e+06   
1921-12-31           2.557507e+06           8.270460e+05   
1922-01-31           4.489400e+06           3.715057e+05   
1922-02-28           5.661319e+06           2.786177e+05   
..                            ...                    ...   
2003-05-31           5.704934e+06           1.099716e+06   
2003-06-30           9.520898e+06           1.231732e+06   
2003-07-31           1.145013e+07           3.030184e+05   
2003-08-31           8.250568e+06           5.057250e+05   
2003-09-30           2.706211e+06           2.198163e+06   

                                                            \
           DEL_CVP_PSC_N_expl0000 DEL_CVP_TOTAL_N_expl0000   
                     DELIVERY-CVP             DELIVERY-CVP   
                             1MON                     1MON   
                         2020D09E                 2020D09E   
                         PER-AVER                 PER-AVER   
                              TAF                      TAF   
1921-10-31           2.041487e+06             3.710914e+06   
1921-11-30           2.603797e+06             3.688032e+06   
1921-12-31           0.000000e+00             7.394474e+05   
1922-01-31           6.411049e+04             5.762657e+05   
1922-02-28           0.000000e+00             5.337932e+05   
..                            ...                      ...   
2003-05-31           1.183967e+07             1.493467e+07   
2003-06-30           1.608900e+07             2.162981e+07   
2003-07-31           1.667051e+07             2.299562e+07   
2003-08-31           1.162066e+07             1.652348e+07   
2003-09-30           3.882223e+06             6.671960e+06   

                                                                               \
           DEL_CVP_TOTAL_S_expl0000 DEL_SWP_MWD_expl0000 DEL_SWP_PMI_expl0000   
                       DELIVERY-CVP         DELIVERY-SWP         DELIVERY-SWP   
                               1MON                 1MON                 1MON   
                           2020D09E             2020D09E             2020D09E   
                           PER-AVER             PER-AVER             PER-AVER   
                      

## Plotting Functions

Need to look example file "general_plots_v20231115.py" is posted to the Google Drive with the example files/scripts in the Python_DSS directory (here: https://drive.google.com/drive/folders/1JbN0eYKNM0772P0XMj0S7Fyl7I9GZxfW?usp=drive_link). Also the plotting.yml file to list out and define which CalSim/CalLite studies to plot and analyze. If you want to test it out yourself, you'll need ot update the paths to your own CalLite/CalSim studies.


In [ ]:
def plot_ts(df, pTitle = 'Time Series', xLab = 'Date', lTitle = 'Studies', fTitle = 'mon_tot', pSave = True, fPath = fPath):
    """
    Plots a time-series graph for a given MultiIndex dataframe (follows calsim conventions)
    
    The function assumes the DataFrame columns follow a specific naming
    convention where the last part of the name indicates the study.
    """
    
    var = '_'.join(df.columns[0][1].split('_')[:-1])
    colormap = plt.cm.tab20
    colors = [colormap(i) for i in range(df.shape[1])]
    colors[-1] = [0,0,0,1]

    count = 0
    
    plt.figure(figsize=(14, 8))
    
    default_font_size = plt.rcParams['font.size']
    scaled_font_size = 1.5 * default_font_size # Change it to font size you want
    default_line_width = plt.rcParams['lines.linewidth']  
    scaled_line_width = 1.5 * default_line_width
    
    studies = [col[1].split('_')[-1] for col in df.columns]

    for study in studies:
        study_cols = [col for col in df.columns if col[1].endswith(study)]
        for col in study_cols:
            sns.lineplot(data=df, x=df.index, y=col, label=f'{study}', color = colors[count], linewidth=scaled_line_width)
            count+=1
            
    plt.title(var + ' ' + pTitle, fontsize=scaled_font_size*2)
    plt.xlabel(xLab, fontsize=scaled_font_size*1.5)
    plt.ylabel(var+"\nUnits: " + df.columns[0][6], fontsize=scaled_font_size*1.5)

    plt.legend(title=lTitle, title_fontsize = scaled_font_size*1.5, fontsize=scaled_font_size*1.25, bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.xticks(rotation=45, fontsize=scaled_font_size)  
    plt.yticks(fontsize=scaled_font_size)  
    plt.tight_layout()  
     
    if pSave:
        plt.savefig(f'{fPath}/{var}_{fTitle}.png', format = 'png', bbox_inches='tight', dpi=600, transparent=False)
        
    plt.show()
   

In [ ]:
def plot_annual_totals(df, xLab = 'Date', pTitle = 'Annual Totals', lTitle = 'Studies', fTitle = 'ann_tot', pSave = True, fPath = fPath):
    """
    Plots a time-series graph of annual totals for a given MultiIndex Dataframe that 
    follows calsim conventions
    
    The function assumes the DataFrame columns follow a specific naming
    convention where the last part of the name indicates the study. 
    """
    
    annualized_df = pd.DataFrame()
    var = '_'.join(df.columns[0][1].split('_')[:-1])
    studies = [col[1].split('_')[-1] for col in df.columns]
        
    colormap = plt.cm.tab20
    colors = [colormap(i) for i in range(df.shape[1])]
    colors[-1] = [0,0,0,1]
        
    i=0

    plt.figure(figsize=(14, 8))
        
    default_font_size = plt.rcParams['font.size']
    scaled_font_size = 1.5 * default_font_size # Change it to font size you want
    default_line_width = plt.rcParams['lines.linewidth']  
    scaled_line_width = 1.5 * default_line_width
    
    for study in studies:
        study_cols = [col for col in df.columns if col[1].endswith(study)]
        for col in study_cols:
            with redirect_stdout(open(os.devnull, 'w')):
                df_ann = csPlots.annualize(df.loc[:, [df.columns[i]]])
                annualized_df = pd.concat([annualized_df, df_ann], axis=1)
                annualized_col_name = df_ann.columns[0]
                sns.lineplot(data = df_ann, x=df_ann.index, y=annualized_col_name, label=f'{study}', color = colors[i],
                            linewidth = scaled_line_width)
                i+=1
                    

    plt.title(var + ' ' + pTitle, fontsize=scaled_font_size*2)
    plt.xlabel(xLab, fontsize=scaled_font_size*1.5)
    plt.ylabel(var+"\nUnits: " + df.columns[0][6], fontsize=scaled_font_size*1.5)

    plt.legend(title=lTitle, title_fontsize = scaled_font_size*1.5, fontsize=scaled_font_size*1.25, bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.xticks(rotation=45, fontsize=scaled_font_size)  
    plt.yticks(fontsize=scaled_font_size)  
    plt.tight_layout()  
        
    if pSave:
        plt.savefig(f'{fPath}/{var}_{fTitle}.png', format = 'png', bbox_inches='tight', dpi=600, transparent=False)
        
    plt.show()
    return annualized_df 

In [ ]:
def plot_exceedance(df, month = "All Months", xLab = 'Probability', pTitle = 'Exceedance Probability', lTitle = 'Studies', fTitle = 'exceed', pSave = True, fPath = fPath):
    """
    Plots an exceedance graph for a given MultiIndex Dataframe that follows calsim conventions
  
    The function assumes the DataFrame columns follow a specific naming
    convention where the last part of the name indicates the study. 
    """
    pTitle = pTitle + " " + month
    fTitle = fTitle + " " + month
    
    var = '_'.join(df.columns[0][1].split('_')[:-1])
    studies = [col[1].split('_')[-1] for col in df.columns]
    i=0
    
    colormap = plt.cm.tab20
    colors = [colormap(i) for i in range(df.shape[1])]
    colors[-1] = [0,0,0,1]

    plt.figure(figsize=(14, 8))
            
    default_font_size = plt.rcParams['font.size']
    scaled_font_size = 1.5 * default_font_size # Change it to font size you want
    default_line_width = plt.rcParams['lines.linewidth']  
    scaled_line_width = 1.5 * default_line_width

    for study in studies:
        study_cols = [col for col in df.columns if col[1].endswith(study)]
        for col in study_cols:
            df_ex = csPlots.single_exceed(df, df.columns[i])
            ex_col_name = df_ex.columns[0]
            sns.lineplot(data = df_ex, x=df_ex.index, y=ex_col_name, label=f'{study}', color = colors[i], linewidth = scaled_line_width)
            i+=1

    plt.title(var + ' ' + pTitle, fontsize=scaled_font_size*2)
    plt.xlabel(xLab, fontsize=scaled_font_size*1.5)
    plt.ylabel(var+"\nUnits: " + df.columns[0][6], fontsize=scaled_font_size*1.5)
    plt.legend(title=lTitle, title_fontsize = scaled_font_size*1.5, fontsize=scaled_font_size*1.25, bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.xticks(rotation=45, fontsize=scaled_font_size)  
    plt.yticks(fontsize=scaled_font_size)  
    plt.tight_layout()  
    
    if pSave:
        plt.savefig(f'{fPath}/{var}_{fTitle}.png', format = 'png', bbox_inches='tight', dpi=600, transparent=False)
        
    plt.show()

In [ ]:
def plot_moy_averages(df, xLab = 'Month of Year', pTitle = 'Month of Year Average Totals', lTitle = 'Studies', fTitle = 'moy_avg', fPath = fPath):
    """
    Plots a time-series graph of month of year averages of a study for a given MultiIndex Dataframe that follows calsim conventions. Calculates mean for 12 months across all study years and uses the plot_ts function to produce a graph.
    
    The function assumes the DataFrame columns follow a specific naming
    convention where the last part of the name indicates the study. 
    """
    df_copy = df.copy()
    df_copy["Month"] = df.index.month
    df_moy = df_copy.groupby('Month').mean()
    plot_ts(df_moy, pTitle = pTitle, xLab = xLab, lTitle = lTitle, fTitle = fTitle, fPath = fPath)

### Difference From Baseline

In [ ]:
def get_difference_from_baseline(df):
    """
    Calculates the difference from baseline for a given variable
    Assumptions: baseline column on first column, df only contains single variable
    """
    df_diff = df.copy()
    baseline_column = df_diff.iloc[:, 0]
    
    for i in range(1, df_diff.shape[1]):
        df_diff.iloc[:, i] = df_diff.iloc[:, i].sub(baseline_column)
    df_diff = df_diff.iloc[:, 1:]

    return df_diff

In [ ]:
def difference_from_baseline(df, plot_type, pTitle = 'Difference from Baseline ', xLab = 'Date', lTitle = 'Studies', fTitle = "___", pSave = True, fPath = fPath):
    """
    Plots the difference from baseline of a single variable with a specific plot type
    plot_type parameter inputs: plot_ts, plot_exceedance, plot_moy_averages, plot_annual_totals
    """
    pTitle += plot_type.__name__
    diff_df = get_difference_from_baseline(df)
    plot_type(diff_df, pTitle = pTitle, fTitle = fTitle, fPath = fPath)

In [ ]:
def slice_with_baseline(df, var, study_lst):
    """
    Creates a subset of df based on varname and slices it according to the provided range.
    """
    subset_df = create_subset(df, var)
    df_baseline = subset_df.iloc[:,[0]]
    df_rest = subset_df.iloc[:, study_lst]
    return pd.concat([df_baseline, df_rest], axis = 1)

In [ ]:
def get_water_year(date):
    date = pd.to_datetime(date)
    return date.year if date.month < 10 else date.year + 1

##### Plots Across WY Type

In [ ]:
def plot_ts_wy(df, titlelabel = '', fPath = '../visualizations', pSave = True):
    """
    Plots time series data for each water year type on the same graph.
    """

    plt.figure(figsize=(10, 6))  
    
    year_types = df["Water Year Type"].unique()

    for year_type in year_types:
        subset = df[df["Water Year Type"] == year_type]
        plt.plot(subset.index, subset.iloc[:, [0]], label=year_type)

    plt.title('Time Series by Water Year Type'+' '+titlelabel)
    plt.xlabel('Date')
    plt.ylabel("Storage (TAF)")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)  
    plt.tight_layout() 
    if pSave:
        plt.savefig(f'{fPath}/TS_WY_{titlelabel}.png', format = 'png', bbox_inches='tight', dpi=600, transparent=False)

    plt.show()

In [ ]:
def plot_moy_wy(df, titlelabel = '', fPath = '../visualizations', pSave = True):
    """
    Plots the month-of-year averages for each water year type.
    """
    
    df['Month'] = df.index.month
    df['Water Year Type'] = df.iloc[:, 1]  
    df_moy = df.groupby(['Month', 'Water Year Type']).mean().reset_index()

    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 6))

    water_year_types = df_moy['Water Year Type'].unique()
    for wy_type in water_year_types:
        subset = df_moy[df_moy['Water Year Type'] == wy_type]
        sns.lineplot(x='Month', y=df.columns[0], data=subset, label=wy_type)

    plt.title('Month-of-Year Averages by Water Year Type'+' '+titlelabel)
    plt.xlabel('Month')
    plt.ylabel('Average Value')
    plt.legend(title='Water Year Type')
    plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
    plt.tight_layout()
    if pSave:
        plt.savefig(f'{fPath}/MoY_WY_{titlelabel}.png', format = 'png', bbox_inches='tight', dpi=600, transparent=False)
    plt.show()

In [ ]:
df_shsta = slice_with_baseline(df, "S_SHSTA", [])
df_orovl = slice_with_baseline(df, "S_OROVL", [])
df_shsta.head(2)

In [ ]:
plot_ts(df_shsta, pTitle = 'Monthly Total', fTitle = 'Month_Tot', fPath = '../visualizations')

In [ ]:
df_del_cvp_pag_n = slice_with_baseline(df, "DEL_CVP_PAG_N", [])
df_del_cvp_pag_s = slice_with_baseline(df, "DEL_CVP_PAG_S", [])
df_del_cvp_pag_n.head(2)

In [ ]:
df_wy_filter.head(2)

In [ ]:
wy_value_map = {
    1: "Wet",
    2: "Above normal",
    3: "Below normal",
    4: "Dry",
    5: "Critical",
}

In [ ]:
water_year_type_map = df_wy_filter.set_index("WATERYEAR")
water_year_type_map = water_year_type_map.to_dict()
water_year_type_map_shasta = water_year_type_map["SACindex_0"]
water_year_type_map_orovl = water_year_type_map["SACindex_0"]
water_year_type_map_del_cvp_pag_n = water_year_type_map["SACindex_0"]
water_year_type_map_del_cvp_pag_s = water_year_type_map["SACindex_0"]

In [ ]:
water_year_type_map_shasta = {year: wy_value_map[value] for year, value in water_year_type_map_shasta.items()}

In [ ]:
water_year_type_map_orovl = {year: wy_value_map[value] for year, value in water_year_type_map_orovl.items()}

In [ ]:
water_year_type_map_del_cvp_pag_n = {year: wy_value_map[value] for year, value in water_year_type_map_del_cvp_pag_n.items()}
water_year_type_map_del_cvp_pag_s = {year: wy_value_map[value] for year, value in water_year_type_map_del_cvp_pag_s.items()}

In [ ]:
df_shsta['Water Year'] = df_shsta.index.map(get_water_year)
df_orovl['Water Year'] = df_orovl.index.map(get_water_year)
df_del_cvp_pag_n['Water Year'] = df_del_cvp_pag_n.index.map(get_water_year)
df_del_cvp_pag_s['Water Year'] = df_del_cvp_pag_s.index.map(get_water_year)

df_shsta['Water Year Type'] = df_shsta["Water Year"].map(water_year_type_map_shasta)
df_orovl['Water Year Type'] = df_orovl["Water Year"].map(water_year_type_map_orovl)
df_del_cvp_pag_n['Water Year Type'] = df_del_cvp_pag_n["Water Year"].map(water_year_type_map_del_cvp_pag_n)
df_del_cvp_pag_s['Water Year Type'] = df_del_cvp_pag_s["Water Year"].map(water_year_type_map_del_cvp_pag_s)

df_shsta = df_shsta.drop(columns = ["Water Year"], axis=1)
df_orovl = df_orovl.drop(columns = ["Water Year"], axis=1)
df_del_cvp_pag_n = df_del_cvp_pag_n.drop(columns = ["Water Year"], axis=1)
df_del_cvp_pag_s = df_del_cvp_pag_s.drop(columns = ["Water Year"], axis=1)

df_shsta.head(5)

In [ ]:
def plot_exceedance_wy(df, titlelabel = '', fPath = '../visualizations', pSave = True):
    """
    Plots an exceedance graph across different water year types.
    """
    wy_types = df['Water Year Type'].unique()
    
    plt.figure(figsize=(12, 6))
    
    for wy_type in wy_types:
        subset = df[df['Water Year Type'] == wy_type]
        
        df_ex = csPlots.single_exceed(subset, subset.columns[0])
        sns.lineplot(x=df_ex.index, y=df_ex.columns[0], data=df_ex, label=wy_type)
    
    plt.title('Exceedance by Water Year Type'+' '+titlelabel)
    plt.xlabel('Exceedance Probability')
    plt.ylabel('Value')
    plt.legend(title='Water Year Type')
    plt.tight_layout()
    if pSave:
        plt.savefig(f'{fPath}/Exc_WY_{titlelabel}.png', format = 'png', bbox_inches='tight', dpi=600, transparent=False)
    plt.show()

In [ ]:
def plot_annualized_wy(df, titlelabel = '', fPath = '../visualizations', pSave = True):
    """
    Plots annualized calculation results across different water year types.
    """
    wy_types = df['Water Year Type'].unique()
    
    plt.figure(figsize=(12, 6))
    
    for wy_type in wy_types:
        subset = df[df['Water Year Type'] == wy_type]
        
        with redirect_stdout(open(os.devnull, 'w')):
        
            df_ann = csPlots.annualize(subset.loc[:, [df.columns[0]]])

            sns.lineplot(x=df_ann.index, y=df_ann.columns[0], data=df_ann, label=wy_type)
    
    plt.title('Annualized Values by Water Year Type'+' '+titlelabel)
    plt.xlabel('Year')
    plt.ylabel('Annualized Value')
    plt.legend(title='Water Year Type')
    plt.tight_layout()
    if pSave:
        plt.savefig(f'{fPath}/Ann_WY_{titlelabel}.png', format = 'png', bbox_inches='tight', dpi=600, transparent=False)
    plt.show()

In [ ]:
def plot_boxplots_wy(df, titlelabel='', pSave = True):
    """
    Creates boxplots for a specified value column, grouped by water year type.
    """
    
    df['Water Year Type'] = pd.Categorical(df['Water Year Type'], categories=df['Water Year Type'].unique(), ordered=True)
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Water Year Type', y=df.columns[0], data=df)

    plt.title(f'Boxplots by Water Year Type {titlelabel}')
    plt.xlabel('Water Year Type')
    plt.ylabel("Value")
    plt.xticks(rotation=45)
    plt.tight_layout()
    if pSave:
        plt.savefig(f'{fPath}/Box_WY_{titlelabel}.png', format = 'png', bbox_inches='tight', dpi=600, transparent=False)
    plt.show()


In [ ]:
#plot_ts_wy(df_shsta, 'Shasta')

In [ ]:
#plot_ts_wy(df_orovl, 'Oroville')

In [ ]:
plot_moy_wy(df_shsta, 'Shasta')

In [ ]:
plot_moy_wy(df_orovl, 'Oroville')

In [ ]:
#plot_annualized_wy(df_shsta, 'Shasta')

In [ ]:
#plot_annualized_wy(df_orovl, 'Oroville')

In [ ]:
# April TS
copy = df_shsta.copy()
apr_shsta = copy[copy.index.month.isin([4])]
#plot_ts_wy(apr_shsta, 'Shasta April Storage')

In [ ]:
plot_exceedance_wy(apr_shsta, 'Shasta April Storage')

In [ ]:
copy = df_shsta.copy()
sep_shsta = copy[copy.index.month.isin([9])]
#plot_ts_wy(sep_shsta, 'Shasta September Storage')

In [ ]:
plot_exceedance_wy(sep_shsta, 'Shasta September Storage')

In [ ]:
# April TS
copy = df_orovl.copy()
apr_orovl = copy[copy.index.month.isin([4])]
#plot_ts_wy(apr_orovl, 'Oroville April Storage')

In [ ]:
plot_exceedance_wy(apr_orovl, 'Oroville September Storage')

In [ ]:
copy = df_orovl.copy()
sep_orovl = copy[copy.index.month.isin([9])]
#plot_ts_wy(sep_orovl, 'Oroville September Storage')

In [ ]:
plot_exceedance_wy(sep_orovl, 'Oroville September Storage')

In [ ]:
plot_moy_wy(df_del_cvp_pag_n, 'del_cvp_pag_n')

In [ ]:
plot_moy_wy(df_del_cvp_pag_s, 'del_cvp_pag_s')

In [ ]:
plot_exceedance_wy(df_del_cvp_pag_n, 'del_cvp_pag_n')

In [ ]:
plot_exceedance_wy(df_del_cvp_pag_s, 'del_cvp_pag_s')

In [ ]:
plot_boxplots_wy(df_shsta, "Shasta")

In [ ]:
plot_boxplots_wy(df_orovl, "Orovl")

In [ ]:
plot_boxplots_wy(df_del_cvp_pag_s, "Del_cvp_pag_s")

In [ ]:
plot_boxplots_wy(df_del_cvp_pag_n, "Del_cvp_pag_n")